In [1]:
import numpy as np
import torch

In [178]:
def group_product(xs, ys):
    """
    the inner product of two lists of variables xs,ys
    :param xs:
    :param ys:
    :return:
    """
    return sum([torch.sum(x * y) for (x, y) in zip(xs, ys)])

def hessian_vector_product(gradsH, params, v):
    """
    compute the hessian vector product of Hv, where
    gradsH is the gradient at the current point,
    params is the corresponding variables,
    v is the vector.
    """
    hv = torch.autograd.grad(gradsH, params, grad_outputs = v, only_inputs = True, retain_graph = True)
    return hv

def normalization(v):
    """
    normalization of a list of vectors
    return: normalized vectors v
    """
    s = group_product(v,v)
#     print("Before", v)
    s = s ** 0.5
    s = s.cpu().item()
    v = [vi / (s + 1e-6) for vi in v]
#     print("After", v)
#     print()
    return v

In [179]:
v = torch.Tensor([[0.3895, 0.2580, -0.5468],
                   [0.7466, 0.3868, 0.5635],
                   [0.5524, 0.8211, -0.5861]])
normalization(v)

[tensor([ 0.2301,  0.1524, -0.3231]),
 tensor([0.4411, 0.2285, 0.3329]),
 tensor([ 0.3264,  0.4851, -0.3463])]

In [ ]:
outputs = model(inputs)
loss = criterion(outputs, targets)
loss.backward(create_graph = True)

params, gradsH = get_params_grad(model)
v = [torch.randn(p.size()).to(device) for p in params]
v = normalization(v)

eigenvalue = None

for i in range(maxIter):
    model.zero_grad()
    Hv = hessian_vector_product(gradsH, params, v)
    eigenvalue_tmp = group_product(Hv, v).cpu().item()
    v = normalization(Hv)
    if eigenvalue == None:
        eigenvalue = eigenvalue_tmp
    else:
        if abs(eigenvalue-eigenvalue_tmp)/abs(eigenvalue) < tol:
            return eigenvalue_tmp, v
        else:
            eigenvalue = eigenvalue_tmp
return eigenvalue, v

In [ ]:
# assume weight already has the grad information
def top_hessian_eigen(weights):
    v = torch.randn(weights.size())
    eigenvalue, eigenvector = None, None
    
    for i in range(1000):
        # outputs will contain graph from inputs to outputs
        # inputs act as a marker in the graph to say where to do backprop until
        # grad_outputs is the gradient wrt to the outputs
            # for example, say we want the gradient of loss wrt to inputsA
            # but we already have gradient of loss wrt to inputsB
            # then gradient of loss wrt inputsA = gradient of loss wrt inputsB * gradients of inputsB wrt inputsA
            # assuming the only path from inputsA to loss passes through inputsB
        Hv = torch.autograd.grad(weights.grad, # outputs
                                 weights, # inputs
                                 grad_outputs=v)

In [82]:
import torch.nn as nn

def reduce(x):
    return (x*x*2).sum()

inputs = torch.tensor([[0, 1],
                       [2, 3.0]], requires_grad=True).float()
outputs = reduce(inputs)
outputs.backward(create_graph=True)

# grad = torch.autograd.grad(outputs, inputs2, retain_graph=True, create_graph=True)
# print(grad)
print(inputs.grad)
print("here")

v = [torch.randn(inputs.size())]
v = normalization(v)

inputs = (inputs)
grads = (inputs.grad)
print("Inputs", inputs)
print("Grads", grads)
print()

eigenvalue = None

tol = 0.000001

for i in range(10000):
#     (Hv,) = torch.autograd.grad(inputs.grad, 
#                                 inputs, 
#                                 grad_outputs=v,
#                                 retain_graph=True)

    print(v)
    Hv = hessian_vector_product(grads, inputs, v)
    eigenvalue_tmp = group_product(Hv, v).cpu().item()
#     print("Hv", Hv)
    print(Hv)
    print()
    v = normalization(Hv)
    if eigenvalue == None:
        eigenvalue = eigenvalue_tmp
    else:
        if abs(eigenvalue-eigenvalue_tmp)/abs(eigenvalue) < tol:
            eigenvalue = eigenvalue_tmp
            break
        else:
            eigenvalue = eigenvalue_tmp

print(eigenvalue, v)

tensor([[ 0.,  4.],
        [ 8., 12.]], grad_fn=<CopyBackwards>)
here
Inputs tensor([[0., 1.],
        [2., 3.]], requires_grad=True)
Grads tensor([[ 0.,  4.],
        [ 8., 12.]], grad_fn=<CopyBackwards>)

[tensor([[ 0.4404,  0.1241],
        [ 0.5275, -0.7158]])]
(tensor([[ 1.7616,  0.4965],
        [ 2.1101, -2.8631]]),)

[tensor([[ 0.4404,  0.1241],
        [ 0.5275, -0.7158]])]
(tensor([[ 1.7616,  0.4965],
        [ 2.1101, -2.8631]]),)

3.9999983310699463 [tensor([[ 0.4404,  0.1241],
        [ 0.5275, -0.7158]])]


In [40]:
J = torch.autograd.functional.jacobian(mse, inputs, create_graph=False, strict=False, vectorize=False)
H = torch.autograd.functional.hessian(mse, inputs, create_graph=False, strict=False, vectorize=False).reshape(4, 4)
print(inputs.grad)
print(J)
print(H)

tensor([-3.1500e+01,  1.1624e+04,  5.9096e+04,  1.9954e+05],
       grad_fn=<CopyBackwards>)
tensor([-3.1500e+01,  1.1624e+04,  5.9096e+04,  1.9954e+05])
tensor([[ 3906.5000,     0.0000,     0.0000,     0.0000],
        [    0.0000, 11680.0000,     0.0000,     0.0000],
        [    0.0000,     0.0000, 29594.5000,     0.0000],
        [    0.0000,     0.0000,     0.0000, 66560.0000]])


In [41]:
import scipy.linalg as la
results = la.eigvals(H.numpy())
results

array([ 3906.5+0.j, 11680. +0.j, 29594.5+0.j, 66560. +0.j],
      dtype=complex64)

In [171]:
import torch                    

def jacobian(y, x, create_graph=False): 
    print(y)
    print(x)
    jac = []                                                                                          
    flat_y = y.reshape(-1)                                                                            
    grad_y = torch.zeros_like(flat_y)                                                                 
    for i in range(len(flat_y)):                                                                      
        grad_y[i] = 1.                                                                                
        grad_x, = torch.autograd.grad(flat_y, x, grad_y, retain_graph=True, create_graph=create_graph)
        jac.append(grad_x.reshape(x.shape))                                                           
        grad_y[i] = 0.                                                                                
    return torch.stack(jac).reshape(y.shape + x.shape)

def hessian(y, x):                                                                                    
    return jacobian(jacobian(y, x, create_graph=True), x)            

print(jacobian(func(layer, y), layer.weight))   
print()
print(hessian(func(layer, y), layer.weight))    

tensor(10., grad_fn=<SumBackward0>)
Parameter containing:
tensor([[0., 1.],
        [2., 3.]], requires_grad=True)
tensor([[0., 2.],
        [0., 6.]])

tensor(10., grad_fn=<SumBackward0>)
Parameter containing:
tensor([[0., 1.],
        [2., 3.]], requires_grad=True)
tensor([[0., 2.],
        [0., 6.]], grad_fn=<ViewBackward>)
Parameter containing:
tensor([[0., 1.],
        [2., 3.]], requires_grad=True)


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [2]] is at version 2; expected version 1 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [177]:
def my_normalize(v):
    factor = (v*v).sum() ** 0.5
    return v / ((v*v).sum() ** 0.5)

x = torch.tensor([[0, 1.0], [2, 3]], requires_grad=True)

layer = torch.nn.Linear(2, 2, bias=False)
layer.weight = torch.nn.parameter.Parameter(x)
y = torch.tensor([0, 5.0], requires_grad=True)


def func(layer, y):
#     m = nn.ReLU()
#     loss = nn.CrossEntropyLoss()
#     target = torch.empty(2, dtype=torch.long).random_(2)
    output = layer(y)
#     return loss(output, target)
#     return layer(y).sum()
    return (output * output).sum()

func(layer, y).backward(create_graph=True)

print("Weight", layer.weight)
print("grad", layer.weight.grad)

v = [torch.Tensor([[-1, -2], [-3, -4.0]])]
v = normalization(v)
print("v", v)

for i in range(10000):
    Hv = hessian_vector_product(layer.weight.grad, layer.weight, v)
    eigenvalue_tmp = group_product(Hv, v).cpu().item()
    v = normalization(Hv)
    if eigenvalue == None:
        eigenvalue = eigenvalue_tmp
    else:
        if abs(eigenvalue-eigenvalue_tmp)/abs(eigenvalue) < tol:
            eigenvalue = eigenvalue_tmp
            break
        else:
            eigenvalue = eigenvalue_tmp

print(eigenvalue, v)

Weight Parameter containing:
tensor([[0., 1.],
        [2., 3.]], requires_grad=True)
grad tensor([[  0.,  50.],
        [  0., 150.]], grad_fn=<CopyBackwards>)
v [tensor([[-0.1826, -0.3651],
        [-0.5477, -0.7303]])]
50.0 [tensor([[ 0.0000, -0.4472],
        [ 0.0000, -0.8944]])]
